In [1]:
import numpy as np 
import re 
from bs4 import BeautifulSoup
import spacy 
import sys
sys.path.append('scripts')
from utility import *
from get_init import *
import json 
import re 
import random 
from spacy.tokens import DocBin
import jsonlines
from spacy.tokens import Span
from prodigy.components.db import connect
from prettytable import PrettyTable
import srsly
import streamlit as st
import streamlit.components.v1 as components
from pysbd.utils import PySBDFactory
from spacy.language import Language
from spacy import displacy
from spacy.pipeline.senter import DEFAULT_SENTER_MODEL
from spacy.language import Language
import pysbd
from spacy.util import filter_spans
from spacy.tokens import Span


#### Extraire les délimiteurs de phrases HTML 

In [2]:

@Language.component("pysbd_sentencizer")
def pysbd_sentence_boundaries(doc):
    seg = pysbd.Segmenter(language="fr", clean=False, char_span=True)
    sents_char_spans = seg.segment(doc.text)
    char_spans = [doc.char_span(sent_span.start, sent_span.end) for sent_span in sents_char_spans]
    start_token_ids = [span[0].idx for span in char_spans if span is not None]
    for token in doc:
        token.is_sent_start = True if token.idx in start_token_ids else False
    return doc

In [3]:
nlp = spacy.load("/Users/anis/test_labnbook/math_ner/train_ner/training/model-best")
nlp.add_pipe("pysbd_sentencizer")

text = "Ceci est une equation : f(x)=x2x1 ? Ceci est une nouvelle phrase de la forme f(x)=ax+b+3 §   Ainsi blablabla  blablabla blablabla $$alpha+X+2$$ blablabla. De plus nous avons la table suivante suivantesuivante suivante suivante ¥ matable ¥ celle-ci www.yahoo-DZ.fr "
doc = nlp(text)
# for sent in doc.sents:
#     print(sent)``
nlp.pipe_names

['tok2vec', 'ner', 'pysbd_sentencizer']

In [4]:
for sent in doc.sents:
    print(sent)

Ceci est une equation : f(x)=x2x1 ? Ceci est une nouvelle phrase de la forme f(x)=ax+b+3 §   Ainsi blablabla  blablabla blablabla $$alpha+X+2$$ blablabla. De plus nous avons la table suivante suivantesuivante suivante suivante ¥ matable ¥ celle-ci www.yahoo-DZ.fr


In [5]:
@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text in ["§"]:
            #print("token.text", token.text)
            doc[token.i + 1].is_sent_start = True
    return doc

nlp.add_pipe("set_custom_boundaries")
doc = nlp(text)
print("After:", [sent.text for sent in doc.sents])
# for token in doc[:-1]:
#     print(token.text, token.i, token.is_sent_start)

After: ['Ceci est une equation : f(x)=x2x1 ? Ceci est une nouvelle phrase de la forme f(x)=ax+b+3 §', '  Ainsi blablabla  blablabla blablabla $$alpha+X+2$$ blablabla. De plus nous avons la table suivante suivantesuivante suivante suivante ¥ matable ¥ celle-ci www.yahoo-DZ.fr']


In [6]:
 for token in doc[:-1]:
        if token.text in ["§","$","¥"]:
            print("token.text", token.text,doc[token.i + 1])

token.text §   
token.text $ $
token.text $ alpha+X+2$$
token.text ¥ matable
token.text ¥ celle-ci


In [7]:
patterns = {
    "LATEX_MATH":  "[$]{1,2}(.*?)[$]{1,2}",
    "TABLE":"¥¥(.*?)¥¥"
        }
@Language.component("Find_Latex-Table")
def find_streets(doc):
    text = doc.text
    camp_ents = []
    original_ents = list(doc.ents)
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, doc.text):
            start, end = match.span()
            span = doc.char_span(start, end)
            if span is not None:
                camp_ents.append((span.start, span.end, span.text))
        for ent in camp_ents:
            start, end, name = ent
            per_ent = Span(doc, start, end, label=label)
            original_ents.append(per_ent)
        filtered = filter_spans(original_ents)
        doc.ents = filtered
    return (doc)
nlp.add_pipe("Find_Latex-Table", before="ner")

<function __main__.find_streets(doc)>

In [12]:
nlp = spacy.load("/Users/anis/test_labnbook/math_ner/correct_ner/training/model-best")
text = "Meubles qui sont encastrables=congélateur. §Meubles qui ne sont pas encastrables=lave-vaisselle, meuble bas et réfrigérateur. §Expliquez par quel raisonnement vous établissez vos conclusions : § J'ai calculé l'intervalle des valeurs que peuvent prendre les meubles dans cet espace en utilisant l'incertitude sur l'espace à meubler. Ainsi on obtient L=[592;602] en utilisant (597+/-5) mm. § Pour le lave vaisselle on a : L=(590+/-1)mm soit un intervalle [589;591] aucune des ces valeurs n'appartiennent à celles de l'intervalle de l'espace. § Pour le congélateur : L=(597+/-1)mm soit [596;598] ces valeurs sont comprises dans l'intervalle de l'espace à meubler donc il est encastrable. § Pour le meuble bas : L=(604+/-1)mm soit [603;605] ces valeurs ne sont pas comprises dans l'intervalle de l'espace meubler, donc il n'est pas encastrable. § Pour le réfrigérateur : L=(592+/-1)mm soit [591;593] ces valeurs ne sont pas totalement comprises dans l'intervalle de l'espace à meubler, donc il n'est pas encastrable. §"
doc = nlp(text)
nlp.pipe_names

['tok2vec', 'ner']

In [13]:
displacy.render(doc,jupyter=True ,style="ent")

In [10]:
displacy.render(doc,jupyter=True ,style="ent")

In [11]:
from add_compoments import *

ModuleNotFoundError: No module named 'add_compoments'

In [ ]:
nlp = spacy.load("/Users/anis/test_labnbook/math_ner/correct_ner/training/model-best-updated")


ValueError: [E002] Can't find factory for 'Find_LatexTable' for language French (fr). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, entity_linker, ner, beam_ner, entity_ruler, lemmatizer, tagger, morphologizer, senter, sentencizer, textcat, spancat, textcat_multilabel, fr.lemmatizer

['tok2vec', 'Find_Latex-Table', 'ner']

In [ ]:
displacy.render(doc,jupyter=True ,style="ent")

In [ ]:
nlp.pipe_labels

{'tok2vec': [], 'ner': ['LATEX_MATH', 'TABLE', 'TEXT_MATH']}

In [ ]:

nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
student1 = nlp(u'LabnBook est une plateforme numérique.')
student2 = nlp(u'LabnBook est une plateforme numérique et gratuite.') 


for token1 in student1:
    for token2 in student2:
        print(token1,"--",token2,"--",token1.similarity(token2))
# student1.similarity(student2)

LabnBook -- LabnBook -- 1.0
LabnBook -- est -- 0.0
LabnBook -- une -- 0.0
LabnBook -- plateforme -- 0.0
LabnBook -- numérique -- 0.0
LabnBook -- et -- 0.0
LabnBook -- gratuite -- 0.0
LabnBook -- . -- 0.0
est -- LabnBook -- 0.0
est -- est -- 1.0
est -- une -- 0.7295531630516052
est -- plateforme -- 0.7295531630516052
est -- numérique -- 0.6456043124198914
est -- et -- 0.5657550096511841
est -- gratuite -- 0.6456043124198914
est -- . -- 0.14166748523712158
une -- LabnBook -- 0.0
une -- est -- 0.7295531630516052
une -- une -- 1.0
une -- plateforme -- 0.9999997615814209
une -- numérique -- 0.812360942363739
une -- et -- 0.572979748249054
une -- gratuite -- 0.812360942363739
une -- . -- 0.033962853252887726
plateforme -- LabnBook -- 0.0
plateforme -- est -- 0.7295531630516052
plateforme -- une -- 0.9999997615814209
plateforme -- plateforme -- 1.0
plateforme -- numérique -- 0.812360942363739
plateforme -- et -- 0.572979748249054
plateforme -- gratuite -- 0.812360942363739
plateforme -- . -- 

/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_48639/1820122094.py:8: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  print(token1,"--",token2,"--",token1.similarity(token2))


In [ ]:
from keybert import KeyBERT



ModuleNotFoundError: No module named 'keybert'